In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

pred_a = pd.read_csv('h2o_no_one_hot_A.csv')
pred_a['date'] = pd.to_datetime(pred_a['date_forecast'])
pred_b = pd.read_csv('h2o_no_one_hot_B.csv')
pred_b['date'] = pd.to_datetime(pred_b['date_forecast'])
pred_c = pd.read_csv('h2o_no_one_hot_C.csv')
pred_c['date'] = pd.to_datetime(pred_c['date_forecast'])

test = pd.read_csv('data/test.csv')
test['time'] = pd.to_datetime(test['time'])

In [3]:
submission = pd.DataFrame(columns=['prediction'])

for val in pred_a['date']:
    if test['time'].eq(val).any():
        row = pred_a.loc[pred_a['date'] == val]
        submission = submission.append({'prediction': float(row['prediction'])}, ignore_index=True)

for val in pred_b['date']:
    if test['time'].eq(val).any():
        row = pred_b.loc[pred_a['date'] == val]
        submission = submission.append({'prediction': float(row['prediction'])}, ignore_index=True)


for val in pred_c['date']:
    if test['time'].eq(val).any():
        row = pred_c.loc[pred_a['date'] == val]
        submission = submission.append({'prediction': float(row['prediction'])}, ignore_index=True)
        
       
submission.to_csv('submission_gb_absolute_error_2.csv')

AttributeError: 'DataFrame' object has no attribute 'append'

In [14]:
def calcGrade(VT_defeated=5):
    number_score = 41 + (100-41)/(10-1) * (VT_defeated-1)
    print(number_score)
    
calcGrade()

67.22222222222223


In [5]:
import utils
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

In [6]:
train_a, test_a = utils.preprocess_category('A')


Total data points: 34085
Data points to be removed: 0


/cluster/home/hansal/ml/utils.py:28: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_interpolated.fillna(method='ffill', inplace=True)
/cluster/home/hansal/ml/utils.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with datetime64[us], please explicitly cast to a compatible dtype first.
  merged_df.fillna(0, inplace=True)


In [7]:
train_a_Y = train_a['pv_measurement']
train_a_X = train_a.drop(columns=['pv_measurement'])
train_a_X = train_a_X.drop(columns=['date_calc', 'date_forecast', 'time'])

In [14]:
import lightgbm as lgb
import optuna
from sklearn.model_selection import cross_val_score

AttributeError: module 'pandas.core.strings' has no attribute 'StringMethods'

In [10]:
def clean_feature_name(name):
    # Replace any special characters you want here
    return name.replace(':', '_').replace('{', '_').replace('}', '_').replace('"', '_')

train_a_X.columns = [clean_feature_name(col) for col in train_a_X.columns]
print(train_a_X)


        absolute_humidity_2m_gm3  air_density_2m_kgm3  ceiling_height_agl_m  \
0                            7.7                1.230           1744.900024   
1                            7.7                1.229           1734.000000   
2                            7.7                1.228           1723.500000   
3                            7.7                1.226           1713.400024   
4                            7.7                1.225           1703.599976   
...                          ...                  ...                   ...   
136237                       4.5                1.281           1532.900024   
136238                       4.5                1.281           1413.199951   
136239                       4.5                1.281           1293.599976   
136240                       4.5                1.281           1173.900024   
136241                       4.5                1.281           1054.199951   

        clear_sky_energy_1h_J  clear_sky_rad_W  clo

In [11]:
def objective(trial):
    dtrain = lgb.Dataset(train_a_X, label=train_a_Y)
    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, step=0.01),
        "boosting_type": trial.suggest_categorical("boosting_type", ['dart', 'gbdt']),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "verbose": -1
    }

    model = lgb.LGBMRegressor(**param)
    model.fit(train_a_X, train_a_Y)

#     result = cross_val_score(model, train_X_std.values, train_Y.values.ravel(), cv=5, scoring="neg_mean_squared_log_error")
    result = cross_val_score(model, train_a_X.values, train_a_Y.values.ravel(), cv=5, scoring="neg_mean_squared_error")


    tot = 0
    for part in result:
        tot = tot + (-part)
    accuracy = tot/len(result)

    return accuracy


In [12]:

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

NameError: name 'optuna' is not defined